In [1]:
from test import *


In [2]:
model, training_losses, training_accuracies, validation_accuracies = train_multi_model(train_data=training_set, test_data=test_set, model=MLP,
                        optim="SAM", batch_size=100, epochs=5, tracking=True)
                        

  0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
print(f"SAM training accuracies: {training_accuracies}")
print(f"SAM validation accuracies: {validation_accuracies}")

SAM training accuracies: [0.7462999820709229, 0.9121666550636292, 0.9282833337783813, 0.9371833205223083, 0.9430333375930786]
SAM validation accuracies: [0.8942999839782715, 0.9240999817848206, 0.9336000084877014, 0.9412999749183655, 0.944100022315979]


In [1]:
# be careful with comparisons, you have to restart the kernel every time to initialize a new model
from test import *

In [2]:
model, training_losses, training_accuracies, validation_accuracies = train_multi_model(train_data=training_set, test_data=test_set, model=MLP,
                        optim="MegaSAM", batch_size=100, epochs=5, tracking=True)

  0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\szilv\OneDrive\Dokumentumok\summer_2022\Summer-2022\MegaAdaptiveSAM\MegaSAM\mega_adaptive_sam.py:91: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2985.)
  norm = torch.sqrt(M_inv.T @ grads_flattened**2)


In [3]:
print(f"SAM training accuracies: {training_accuracies}")
print(f"SAM validation accuracies: {validation_accuracies}")

SAM training accuracies: [0.4179333448410034, 0.8238499760627747, 0.8766833543777466, 0.8924166560173035, 0.9017500281333923]
SAM validation accuracies: [0.7551000118255615, 0.8705999851226807, 0.8917999863624573, 0.9021999835968018, 0.9074000120162964]
